In [1]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from mido import MidiFile ,MetaMessage, Message, MidiTrack, second2tick
from models.LSTM_SELFTRY_Minimized import create_network, sample_with_temp

import matplotlib.pyplot as plt


Using TensorFlow backend.


# parameters

In [2]:
# run params
section = 'compose'
run_id = '1120'
music_name = 'midis'
run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

# model params
embed_size = 100
rnn_units = 256
use_attention = True


## load the lookup tables

In [3]:

store_folder = os.path.join(run_folder, 'store')

with open(os.path.join(store_folder, 'distincts'), 'rb') as filepath:
    distincts = pkl.load(filepath)
    command_names, n_commands, channel_names, n_channel, value1_names, n_value1, duration_names, n_durations = distincts

with open(os.path.join(store_folder, 'lookups'), 'rb') as filepath:
    lookups = pkl.load(filepath)
    command_to_int, int_to_command, channel_to_int, int_to_channel, value1_to_int, int_to_value1, duration_to_int, int_to_duration = lookups

## build the model

In [4]:
weights_folder = os.path.join(run_folder, 'weights')
weights_file = 'weights_custom.h5'
#weights_file = 'weights-improvement-138-5.0825-bigger_custom.h5'

model, att_model = create_network(n_commands, n_channel, n_value1, n_durations, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
commands_in (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
channels_in (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
values1_in (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
durations_in (InputLayer)       [(None, None)]       0                                            
______________________________________________________________________________________________

## build your own phrase

In [5]:
# prediction params
commands_temp = 0.5
channel_temp = 0.5
values1_temp = 0.5
duration_temp = 0.5

max_extra_commands = 150
max_seq_len = 128
sequence_length = 64

commands = ['START']
channels = [-1]
values1 = [-1]
durations = [0]


## Generate notes from the neural network based on a sequence of notes

In [6]:
def generateMidoMessage(command, channel, value1, duration):
    value1 = int(value1)    
    duration = int(duration)
    pChannel = int(channel)
    
    if command == 'note_on' or command == 'note_off':
        # Use default velocity
        return Message(command, channel=pChannel, note=value1, velocity=128, time=duration)
    elif command == 'program_change':
        return Message(command, channel=pChannel, program=value1, time=duration)

In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

prediction_output = []

default_tempo = 500000
default_ticks_per_second = 480

commands_input_sequence = []
channel_input_sequence = []
value1_input_sequence = []
durations_input_sequence = []

for co, ch, v1, d in zip(commands, channels, values1, durations):
    command_int = command_to_int[co]
    channel_int = channel_to_int[ch]
    value1_int = value1_to_int[str(v1)]
    duration_int = duration_to_int[d]
    
    commands_input_sequence.append(command_int)
    channel_input_sequence.append(channel_int)
    value1_input_sequence.append(value1_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([co, ch, v1, d])
    
    if co == 'START':
        createdMidi = MidiFile()
        track = MidiTrack()
        createdMidi.tracks.append(track)
        # Set ticks_per_second of Midi to default
        createdMidi.ticks_per_second = default_ticks_per_second
        # Set Tempo of Midi to default
        track.append(MetaMessage('set_tempo', tempo=default_tempo, time=0))
    elif co == 'programm_change':
        track.append(Message(co, channel=ch, program=v1, time=0))
    elif co == 'note_on' or co == 'note_off':
        pTime = int(second2tick(d, ticks_per_beat=default_ticks_per_second, tempo=default_tempo))
        track.append(Message(co, channel=ch, note=v1, time=pTime))
    else:
        print('Error in else!')


att_matrix = np.zeros(shape = (max_extra_commands+sequence_length, max_extra_commands))

for command_index in range(max_extra_commands):

    print(command_index)
    prediction_input = [
        np.asarray([commands_input_sequence])
        , np.asarray([channel_input_sequence])
        , np.asarray([value1_input_sequence])
        , np.asarray([durations_input_sequence])
       ]
    command_prediction, channel_prediction, value1_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    if use_attention:
        att_prediction = att_model.predict(prediction_input, verbose=0)[0]
        att_matrix[(command_index-len(att_prediction)+sequence_length):(command_index+sequence_length), command_index] = att_prediction
            
    i1 = sample_with_temp(command_prediction[0], commands_temp)
    i2 = sample_with_temp(channel_prediction[0], channel_temp)
    i3 = sample_with_temp(value1_prediction[0], values1_temp)
    i4 = sample_with_temp(durations_prediction[0], duration_temp)    

    command_result = int_to_command[i1]
    channel_result = int_to_channel[i2]
    value1_result = int_to_value1[i3]
    duration_result = int_to_duration[i4]
    
    print(command_result, channel_result, value1_result, duration_result)
    
    prediction_output.append([command_result, channel_result, value1_result, duration_result])

    commands_input_sequence.append(i1)
    channel_input_sequence.append(i2)
    value1_input_sequence.append(i3)
    durations_input_sequence.append(i4)
    
    if len(commands_input_sequence) > max_seq_len:
        commands_input_sequence = commands_input_sequence[1:]
        channel_input_sequence = channel_input_sequence[1:]
        value1_input_sequence = value1_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
        
    if command_result == 'START':
        break
        
print('Generated sequence of {} notes'.format(len(prediction_output)))

In [11]:
# Temporarily diasble #### Remove
#fig, ax = plt.subplots(figsize=(15,15))
#ax.set_yticks([int(j) for j in range(35,70)])

#plt.imshow(overall_preds[35:70,:], origin="lower", cmap='coolwarm', vmin = -0.5, vmax = 0.5, extent=[0, max_extra_notes, 35,70])
#################################

## convert the output from the prediction to notes and create a midi file from the notes 

In [9]:
output_folder = os.path.join(run_folder, 'output')

# create note and chord objects based on the values generated by the model
print(prediction_output)
for pattern in prediction_output:
    command_pattern, channel_pattern, value1_pattern, duration_pattern = pattern
    pTime = int(second2tick(duration_pattern, ticks_per_beat=default_ticks_per_second, tempo=default_tempo))
    # pattern is a chord
    if command_pattern == 'START':
        print('START')
        continue
    elif command_pattern == 'program_change':
        print('program_change')
        track.append(Message(command_pattern, channel=channel_pattern, program=int(value1_pattern), time=pTime))
        continue
    elif command_pattern == 'note_on' or command_pattern == 'note_off':
        print(command_pattern)
        track.append(Message(command_pattern, channel=channel_pattern, note=int(value1_pattern), time=pTime))
        continue

timestr = time.strftime("%Y%m%d-%H%M%S")
createdMidi.save(os.path.join(output_folder, 'output-minloss-' + timestr + '.mid'))

[['START', -1, -1, 0], ['note_off', 14, '30', 0.1], ['note_on', 10, '63', 0], ['note_on', 6, '47', 0], ['START', 7, '66', 0.3]]
START
note_off
note_on
note_on
START
